In [5]:
import torch
import torchvision
import numpy as np
import pandas as pd
import argparse
import os
import copy
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from PIL import Image
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

LABELS_Severity = {35: 0, 43: 0, 47: 1, 53: 1, 61: 2, 65: 2, 71: 2, 85: 2}

mean = (.1706)
std = (.2112)

normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor(),
    normalize,
])

#Check if GPU is being used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

#Define the dataloader class
class OCTDataset(Dataset):
    def __init__(self, subset='train', transform=None):
        if subset == 'train':
            self.annot = pd.read_csv('df_prime_train.csv')
        elif subset == 'test':
            self.annot = pd.read_csv('df_prime_test.csv')
        self.annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(self.annot['DRSS'].values)]
        self.root = os.path.expanduser('/storage/home/hpaceice1/shared-classes/materials/ece8803fml/')
        self.transform = transform
        self.nb_classes=len(np.unique(list(LABELS_Severity.values())))
        self.path_list = self.annot['File_Path'].values
        self._labels = self.annot['Severity_Label'].values
        assert len(self.path_list) == len(self._labels)

    def __getitem__(self, index):
        img, target = Image.open(self.root+self.path_list[index]).convert("L"), self._labels[index]
        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self._labels)         

Using cuda device


In [6]:
#Define AlexNet Model
def AlexNetClassifier():
    
    model = torchvision.models.alexnet()
    model.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
        
    # Replace the last layer
    num_classes = 3
    model.classifier[6] = nn.Linear(4096, num_classes)
    
    model = model.to('cuda')
    
    return model

In [7]:
#Define the model and the training loop
Alex_Model = AlexNetClassifier()

# Define hyperparameters: Batch Size, Learning Rate, Number of Epochs
batch_size = 100
lr = 0.0001
num_epochs = 20

# Define loss function and optimizer
Loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Alex_Model.parameters(), lr=lr)
epoch = 0

def train_AlexNet(train_loader):
    Alex_Model.train()
    # Training loop
    for epoch in range(num_epochs):
        # Train the CNN on the images in the training set
        for i, (images, labels) in enumerate(train_loader):
            images = images.to('cuda') 
            labels = labels.to('cuda')
            optimizer.zero_grad()
            outputs = Alex_Model(images)
            loss = Loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
         
        print('Epoch: {} | Train loss: {:0.4f}'.format(epoch, loss.item()))

In [8]:
#Create training & testing sets
trainset = OCTDataset( 'train', transform=transform)
testset = OCTDataset( 'test', transform=transform)
print('Successfully created training and testing sets')

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)
print('Successfully created the Dataloaders')

#Train the ALexNet model
train_AlexNet(train_loader)

Successfully created training and testing sets
Successfully created the Dataloaders
Epoch: 0 | Train loss: 0.7068
Epoch: 1 | Train loss: 0.5133
Epoch: 2 | Train loss: 0.6089
Epoch: 3 | Train loss: 0.4136
Epoch: 4 | Train loss: 0.4690
Epoch: 5 | Train loss: 0.4128
Epoch: 6 | Train loss: 0.3927
Epoch: 7 | Train loss: 0.3284
Epoch: 8 | Train loss: 0.2819
Epoch: 9 | Train loss: 0.2350
Epoch: 10 | Train loss: 0.3248
Epoch: 11 | Train loss: 0.1207
Epoch: 12 | Train loss: 0.1994
Epoch: 13 | Train loss: 0.2309
Epoch: 14 | Train loss: 0.1083
Epoch: 15 | Train loss: 0.1628
Epoch: 16 | Train loss: 0.1460
Epoch: 17 | Train loss: 0.1437
Epoch: 18 | Train loss: 0.1749
Epoch: 19 | Train loss: 0.1702


In [9]:
#Training & Testing Set performances
def evaluate_loss(model, data_loader):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to('cuda')
            labels = labels.to('cuda')
            outputs = model(images)
            loss = Loss_fn(outputs, labels)
            total_loss += loss.item() * images.size(0)
            num_batches += 1
    avg_loss = total_loss / len(data_loader.dataset)
    return avg_loss

# Compute training and testing loss
train_loss = evaluate_loss(Alex_Model, train_loader)
test_loss = evaluate_loss(Alex_Model, test_loader)

print('Training Loss: {:.4f}'.format(train_loss))
print('Testing Loss: {:.4f}'.format(test_loss))

Training Loss: 0.1005
Testing Loss: 4.1615


In [10]:
# Evaluate the model on the test set
#Compute different performance metrics
Alex_Model.eval()
with torch.no_grad():
    test_acc = 0.0
    test_f1 = 0.0
    y_true = []
    y_pred = []
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = Alex_Model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.tolist())
        y_pred.extend(predicted.tolist())
        test_acc = balanced_accuracy_score(y_true, y_pred)
            
    # Print training progress
    print(f"Test Balanced Accuracy Score:{test_acc:.4f}")

/storage/home/hpaceice1/shussain81/.conda/envs/ece8803/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/storage/home/hpaceice1/shussain81/.conda/envs/ece8803/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/storage/home/hpaceice1/shussain81/.conda/envs/ece8803/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/storage/home/hpaceice1/shussain81/.conda/envs/ece8803/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/storage/home/hpaceice1/shussain81/.conda/envs/ece8803/lib/python3.8/sit

Test Balanced Accuracy Score:0.3353


In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score

# y_test and y_pred are defined as in the previous example

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Compute accuracy and balanced accuracy scores
acc = accuracy_score(y_true, y_pred)
bacc = balanced_accuracy_score(y_true, y_pred)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Compute precision, recall, and f1-score
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Compute true positive rate (recall) and false positive rate (1 - specificity)
if cm.shape == (2, 2):  # binary classification
    tn, fp, fn, tp = cm.ravel()
else:  # multiclass classification
    tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)


print("Confusion matrix:")
print(cm)
print("Accuracy: {:.4f}".format(acc))
print("Balanced accuracy: {:.4f}".format(bacc))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("True positive rate (recall): {:.4f}".format(tpr))
print("False positive rate (1 - specificity): {:.4f}".format(fpr))

Confusion matrix:
[[ 809 1275  464]
 [ 949 2115  856]
 [ 733  560  226]]
Accuracy: 0.3944
Balanced accuracy: 0.3353
Precision: 0.3355
Recall: 0.3353
F1-score: 0.3354
True positive rate (recall): 0.6903
False positive rate (1 - specificity): 0.6118
